<a href="https://www.kaggle.com/code/tridibraj/dcgan-model-for-face-generation-need-improvement?scriptVersionId=202680434" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Create a small Dataset

In [ ]:
import os
import random
import shutil

# Constants
SOURCE_DIR = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba'
TARGET_DIR = '/kaggle/working/selected_images'
NUM_IMAGES = 20000

# Ensure the target directory exists
os.makedirs(TARGET_DIR, exist_ok=True)

# List all image files in the source directory
all_image_files = [f for f in os.listdir(SOURCE_DIR) if f.endswith(('.jpg', '.png'))]

# Randomly select a subset of images
selected_files = random.sample(all_image_files, NUM_IMAGES)

# Copy selected images to the target directory
for filename in selected_files:
    src_path = os.path.join(SOURCE_DIR, filename)
    dst_path = os.path.join(TARGET_DIR, filename)
    shutil.copyfile(src_path, dst_path)

print(f"Copied {NUM_IMAGES} images to {TARGET_DIR}")


# Delete Files if Needed

In [ ]:
# import os
# import shutil

# # Path to the kaggle working directory
# WORKING_DIR = '/kaggle/working'

# # List all items in the working directory
# items = os.listdir(WORKING_DIR)

# # Iterate over all items
# for item in items:
#     item_path = os.path.join(WORKING_DIR, item)
#     # Check if the item is a directory
#     if os.path.isdir(item_path):
#         print(f"Deleting folder: {item_path}")
#         shutil.rmtree(item_path)  # Delete the folder and its contents

# print("All folders in the kaggle working directory have been deleted.")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image, make_grid
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# Loading Dataset

In [ ]:


# Constants
DATA_DIR = '/kaggle/working/selected_images'  # Use the kaggle/working directory
IMAGE_SIZE = 64
BATCH_SIZE = 128
LATENT_DIM = 100
EPOCHS = 300
SAVE_INTERVAL = 3  # Interval to save and show images

# Create directory to save images
os.makedirs("images", exist_ok=True)

# Transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Custom dataset to load images from a list of file paths
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith(('.jpg', '.png'))]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image

print("Loading dataset...")
# Create the dataset and data loader for the images
dataset = ImageDataset(DATA_DIR, transform=transform)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
print("Dataset loaded and data loader created.")

# Architecture

In [ ]:

# Define the Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(LATENT_DIM, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256, 0.8),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512, 0.8),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024, 0.8),
            nn.ReLU(),
            nn.Linear(1024, IMAGE_SIZE * IMAGE_SIZE * 3),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 3, IMAGE_SIZE, IMAGE_SIZE)
        return img

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(IMAGE_SIZE * IMAGE_SIZE * 3, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Initialize the models
generator = Generator()
discriminator = Discriminator()

# Loss and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


# Train

In [ ]:
# Function to display images
def display_images(epoch):
    z = torch.randn(25, LATENT_DIM)
    gen_imgs = generator(z)
    gen_imgs = gen_imgs.view(25, 3, IMAGE_SIZE, IMAGE_SIZE)
    gen_imgs = gen_imgs * 0.5 + 0.5  # Denormalize
    grid = make_grid(gen_imgs, nrow=5)
    np_grid = grid.numpy()
    plt.figure(figsize=(10, 10))
    plt.imshow(np.transpose(np_grid, (1, 2, 0)))
    plt.title(f"Epoch {epoch}")
    plt.show()

# Training the GAN
print("Starting training...")
for epoch in range(EPOCHS):
    for i, imgs in enumerate(data_loader):
        
        # Ground truths
        valid = torch.ones(imgs.size(0), 1, requires_grad=False)
        fake = torch.zeros(imgs.size(0), 1, requires_grad=False)
        
        # Configure input
        real_imgs = imgs.type(torch.FloatTensor)
        
        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(imgs.shape[0], LATENT_DIM)
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()
        
        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        
        # Print progress
        if i % 500 == 0:
            print(f"[Epoch {epoch}/{EPOCHS}] [Batch {i}/{len(data_loader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    # Save and show generated samples after every 3 epochs
    if epoch % SAVE_INTERVAL == 0:
        print(f"Displaying and saving images for epoch {epoch}...")
        display_images(epoch)
        save_image(gen_imgs.data[:25], f"images/epoch_{epoch}.png", nrow=5, normalize=True)

# Save the generator model
print("Training complete. Saving the generator model...")


# Save the Model

In [ ]:
# Initialize the Generator model
generator = Generator()

# Save the generator model to the Kaggle working directory
torch.save(generator.state_dict(), "/kaggle/working/generator.pth")
print("Model saved to /kaggle/working/generator.pth")


In [ ]:
import os

def find_file(root_dir, filename):
    for root, dirs, files in os.walk(root_dir):
        if filename in files:
            return os.path.join(root, filename)
    return None

# Define the root directory and the filename
root_dir = '/kaggle/working'
filename = 'generator.pth'

# Find the file and print the location
file_path = find_file(root_dir, filename)

if file_path:
    print(f"File '{filename}' found at: {file_path}")
else:
    print(f"File '{filename}' not found in {root_dir}")


# Modified architecture ( takes too much time to train)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image, make_grid
from PIL import Image
from torch.utils.data import Dataset, DataLoader


In [ ]:

# Constants
DATA_DIR = '/kaggle/working/selected_images'  # Use the kaggle/working directory
IMAGE_SIZE = 64
BATCH_SIZE = 128
LATENT_DIM = 100
EPOCHS = 3
SAVE_INTERVAL = 3  # Interval to save and show images

# Create directory to save images
os.makedirs("images", exist_ok=True)

# Transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Custom dataset to load images from a list of file paths
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.file_list = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith(('.jpg', '.png'))]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image

print("Loading dataset...")
# Create the dataset and data loader for the images
dataset = ImageDataset(DATA_DIR, transform=transform)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
print("Dataset loaded and data loader created.")

In [ ]:
# Define the Generator (DCGAN)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = IMAGE_SIZE // 4
        self.l1 = nn.Sequential(nn.Linear(LATENT_DIM, 128 * self.init_size ** 2))
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# Define the Discriminator (DCGAN)
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
            nn.Conv2d(16, 32, 3, 2, 1),
            nn.BatchNorm2d(32, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
            nn.Conv2d(32, 64, 3, 2, 1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(0.25),
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

# Initialize the models
generator = Generator()
discriminator = Discriminator()

# Loss and optimizers
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Function to display images
def display_images(epoch):
    z = torch.randn(25, LATENT_DIM)
    gen_imgs = generator(z)
    gen_imgs = gen_imgs * 0.5 + 0.5  # Denormalize
    grid = make_grid(gen_imgs, nrow=5)
    np_grid = grid.numpy()
    plt.figure(figsize=(10, 10))
    plt.imshow(np.transpose(np_grid, (1, 2, 0)))
    plt.title(f"Epoch {epoch}")
    plt.show()




In [ ]:
# Training the GAN
print("Starting training...")
for epoch in range(EPOCHS):
    for i, imgs in enumerate(data_loader):
        
        # Ground truths
        valid = torch.ones(imgs.size(0), 1, requires_grad=False)
        fake = torch.zeros(imgs.size(0), 1, requires_grad=False)
        
        # Configure input
        real_imgs = imgs.type(torch.FloatTensor)
        
        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(imgs.shape[0], LATENT_DIM)
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()
        
        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        
        # Print progress
        if i % 500 == 0:
            print(f"[Epoch {epoch}/{EPOCHS}] [Batch {i}/{len(data_loader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

    # Save and show generated samples after every 3 epochs
    if epoch % SAVE_INTERVAL == 0:
        print(f"Displaying and saving images for epoch {epoch}...")
        display_images(epoch)
        save_image(gen_imgs.data[:25], f"images/epoch_{epoch}.png", nrow=5, normalize=True)

In [ ]:
# Save the generator model
print("Training complete. Saving the generator model...")
torch.save(generator.state_dict(), "generator.pth")
print("Model saved.")
